In [ ]:
# =========================================== BASIC DATAFRAME FUNCTIONS ===========================================

# ExtractLayer takes a chipID and returns its half and layer
def ExtractHalfLayer(ChipID):
    return Chip_DF.at[ChipID+1, 'Half'], Chip_DF.at[ChipID,'Layer']

# ExtractLayer takes a chipID and returns its layer
def ExtractLayer(ChipID):
    return int(Chip_DF.at[ChipID+1,'Layer'][5])

# BuildLayerDF returns a sub-dataframe of Chip_DF with the chips on "layer"
def BuildLayerDF(layer, DF = Chip_DF):
    DF_name = layer
    Sub_DF = DF[(Chip_DF['Layer'] == layer)]
    return Sub_DF

# BuildHalfLayerDF returns a sub-dataframe of Chip_DF with the chips on "layer" and "half"
def BuildHalfLayerDF(half,layer):
    DF_name = half + '_' + layer
    Sub_DF = Chip_DF[(Chip_DF['Half'] == half) & (Chip_DF['Layer'] == layer)]
    return [DF_name, Sub_DF]

# This functions builds a dataframe for a single 'chip1' on layer9 with all the chips in the cone generated by 'chip1' with their overlap
def BuildOverlapDF(chip=0,chipID=[], layer='0', X=0, Y=0, Z=0, Overlap=0):
    d = {'Layer9 Chip':chip,'ChipID': chipID, 'Layer':layer, 'X':X, 'Y':Y, 'Z':Z, 'Overlap':Overlap}
    return pd.DataFrame(data=d)
    
def BuildPEffDF(ListChipID, ListHalf, L0Eff=1, L1Eff=1, L2Eff=1, L3Eff=1, L4Eff=1, 
                L5Eff=1, L6Eff=1, L7Eff=1, L8Eff=1):
    Peff = L0Eff*L1Eff*L2Eff*L3Eff*L4Eff*L5Eff*L6Eff*L7Eff*L8Eff
    Data = {'ChipID':ListChipID, 'Half':ListHalf, 'Layer0':L0Eff, 'Layer1':L1Eff, 'Layer2':L2Eff, 'Layer3':L3Eff, 
            'Layer4':L4Eff, 'Layer5':L5Eff, 'Layer6':L6Eff, 'Layer7':L7Eff, 'Layer8':L8Eff, 'Pseaudo_Eff':Peff}
    return pd.DataFrame(data=Data)

# Creates a directory in the input path if it does not exist
def CreateDirectory(directory_name):
    if not os.path.exists(directory_name):
        os.makedirs(directory_name)

# =========================================== PLOTTING FUNCTIONS ===========================================

def Draw_Graph(X, Y, title, xtitle, ytitle):
    graph = ROOT.TGraph()

    # Fill the graph with data points
    for i in range(0,len(X)):
        graph.SetPoint(i, X[i], Y[i]) 

    # Set graph marker properties
    graph.SetMarkerStyle(ROOT.kFullCircle)
    graph.SetMarkerSize(1)

    # Set axis labels
    graph.GetXaxis().SetTitle(xtitle)
    graph.GetYaxis().SetTitle(ytitle)

    # Set axis range
    x_min = min(X) - 1
    x_max = max(X) + 1
    y_min = min(Y) - 10
    y_max = max(Y) + 10
    graph.GetXaxis().SetLimits(x_min, x_max)
    graph.GetYaxis().SetRangeUser(y_min, y_max)

    return graph

def BuildHist1(name, nb_bin, x, y, titles):
    Hist_name = ROOT.TH1F(name, name, nb_bin, x, y)
    X = Hist_name.GetXaxis()
    X.SetTitle(titles[0]) #define axis label
    Y = Hist_name.GetYaxis()
    Y.SetTitle(titles[1])
    return [Hist_name, X, Y]

def BuildHist(titles):
    Hist = ROOT.TH1F()
    X = Hist.GetXaxis()
    X.SetTitle(titles[0]) #define axis label
    Y = Hist.GetYaxis()
    Y.SetTitle(titles[1])
    return Hist

# DrawList draws the elements of an input list on a ROOT TCanvas
def DrawList(l): 
    for i in range(len(l)):
        l[i].Draw()
        
# ==> DRAWING FUNCTIONS SPECIFIC TO CUSTOM 2D HISTOGRAM



def Corners(chipID):
    x,y,z = GetCoor(chipID)[0], GetCoor(chipID)[1], GetCoor(chipID)[2]
    corner1 = np.array([x,y,z])
    corner2 = np.array([x-DimChip[0], y, z])
    corner3 = np.array([x-DimChip[0], y-DimChip[1], z])
    corner4 = np.array([x, y-DimChip[1], z])
    return np.array([corner1, corner2, corner3, corner4])

def ChipLimits(x,y, t=1):
    return int(x-DimChip[0]*t), int(y-DimChip[1]*t)

def DrawRectangle(X, Y, t=1):
    array = np.zeros((dim1,dim2))
    #Xlim, Ylim = ChipLimits(X,Y,t)
    Xlim, Ylim = int(X-DimChip[0]*t), int(Y-DimChip[1]*t)
    array[Xlim:X + 1, Ylim:Y + 1] = 1
    #array[Ylim:Y + 1, Xlim:X + 1] = 1
    array = np.rot90(array, k=1)
    #array = np.flip(array, axis=1)
    return array

# Returns the new x,y coordinates of a point that is on a line from IP to a point (xa,ya,za)
def PointOnLine(xa,ya,za,zb):
    t = zb/za
    x = t*xa
    y = t*ya
    return x,y, t

def SaveAsImage(array, file_path):
    # Convert the array to a numpy array
    numpy_array = np.array(array)
    # Convert the numpy array to a PIL image
    image = Image.fromarray((numpy_array * 255).astype('uint8'), 'L')
    # Save the image
    image.save(file_path)

# Testing if there are at least 2 clusters on D3 and/or D4
def Sub_Condition(layers):
    if 6 in layers and 7 in layers:
        return True
    elif 6 in layers and 8 in layers:
        return True
    elif 6 in layers and 9 in layers:
        return True
    elif 7 in layers and 8 in layers:
        return True
    elif 7 in layers and 9 in layers:
        return True
    elif 8 in layers and 9 in layers:
        return True
    else:
        return False
    
def LayerCombinations(n_cluster):
    Layers = ['layer0', 'layer1', 'layer2', 'layer3', 'layer4', 'layer5','layer6', 'layer7', 'layer8','layer9']
    """
    Generate all possible lists of length n_cluster from the elements of Layers
    where at least 4 disks are represented and there are at least 2 clusters on D3 and/or D4
    """
    # Generate all combinations
    all_combinations = [list(comb) for comb in combinations(Layers, n_cluster)]
    
    valid_combinations = []
    for combination in all_combinations:
        disks = set()
        config = []
        for layer in combination:
            l = int(layer[-1])
            config.append(l)
            disk_number = l // 2
            disks.add(disk_number)
        if len(disks) >= 4 and len(combination)>=5 and Sub_Condition(config):
            valid_combinations.append(combination)

    return valid_combinations

def LayerCombinations_STANDARD(n_cluster):
    Layers = ['layer0', 'layer1', 'layer2', 'layer3', 'layer4', 'layer5','layer6', 'layer7', 'layer8','layer9']
    """
    Generate all possible lists of length n_cluster from the elements of Layers
    where at least 4 disks are represented.
    """
    # Generate all combinations
    all_combinations = [list(comb) for comb in combinations(Layers, n_cluster)]
    
    # Filter combinations where at least 4 disks are represented
    valid_combinations = []
    for combination in all_combinations:
        disks = set()
        for layer in combination:
            disk_number = int(layer[5:]) // 2
            disks.add(disk_number)
        if len(disks) >= 4 and len(combination)>=5:
            valid_combinations.append(combination)
    
    return valid_combinations
        
def DrawPeffVsConfigs(n_clusters,zIP=0):
    Configs = LayerCombinations(n_clusters)
    
    ListID = Chip_DF['ChipID'].values

    for chip in ListID:
        #print(" For Chip {} :".format(chip))
        Pseaudo_Eff = []
        Config_Names = []
        Config_Paths = []
        for config in Configs:
            Config_Name = '/Config_' + '_'.join(layer[5] for layer in config)
            Config_Path = f"Peff_Results/z_{zIP}/{n_clusters}_Clusters{Config_Name}"
            
            if get_pseaudo_eff(chip,Config_Path+"/PeffDF.txt")>0:
                Config_Names.append(Config_Name[8:])
                Config_Paths.append(Config_Path)
                Pseaudo_Eff.append(get_pseaudo_eff(chip, Config_Path+"/PeffDF.txt"))
                
        if len(Config_Names) != 0:
            Pseaudo_Eff = np.array(Pseaudo_Eff)*100
            Pseaudo_Eff_mean = np.mean(Pseaudo_Eff)        
            
            fig = plt.figure(figsize=(max(12, 10+int(len(Config_Names)/5)),10))
            ax = fig.add_subplot(111)
            ax.grid()
            ax.title.set_text(f'Pseaudo Efficiency vs layer configuration for chip {chip}')
            ax.set_xlabel('Configuration')
            ax.set_ylabel('Pseaudo Efficiency (%)')
            ax.set_ylim(0, 60)
            ax.tick_params(axis='x', labelrotation=45)
            ax.scatter(Config_Names, Pseaudo_Eff)
            ax.plot(Config_Names, np.ones(len(Config_Names))*Pseaudo_Eff_mean, color='red', label='Mean')
            ax.legend()
            CreateDirectory(f"Peff_Results/z_{zIP}/{n_clusters}_Clusters/Chips")
            plt.savefig(f"Peff_Results/z_{zIP}/{n_clusters}_Clusters/Chips/{chip}.jpg")

            #print(f"Saving figure: Peff_Results/z_{zIP}/{n_clusters}_Clusters/Chips/{chip}.jpg)
            
        plt.close()

def get_pseaudo_eff(chip_id, file_path):
    """
    Get the value under 'Pseaudo_Eff' corresponding to the given ChipID from the text file.
    """
    with open(file_path, 'r') as file:
        next(file)  # Skip the header line
        for line in file:
            data = line.strip().split()
            
            if data[0] == str(chip_id):
                return float(data[-1])  # Return the value under 'Pseaudo_Eff'
    # If ChipID is not found
    return 0

def get_weight(config, file_path):
    with open(file_path, 'r') as file:
        next(file)
        for line in file:
            data = line.strip().split()
            if data[0] == config:
                return float(data[-1])

def ChangeToEtaPhi2(x,y,z):
    x = x - 0.65
    y = y - 1.5

    rxy = np.sqrt(x**2 + y**2 + z**2)
    theta = np.arccos(z/rxy)
    
    signy = m.copysign(1,y)
    phi = signy*np.arccos(x/np.sqrt(x**2 + y**2))
    eta = (-np.log(np.tan(theta/2))).tolist()

    return eta, phi

def ChangeToEtaPhi(chipID):
    x,y,z = GetTrueCoor(chipID+1)[0], GetTrueCoor(chipID+1)[1], GetTrueCoor(chipID+1)[2]

    rxy = np.sqrt(x**2 + y**2 + z**2)
    theta = np.arccos(z/rxy)
    
    signy = m.copysign(1,y)
    phi = signy*np.arccos(x/np.sqrt(x**2 + y**2))
    eta = (-np.log(np.tan(theta/2))).tolist()

    return eta, phi